In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


In [2]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [3]:
num_classes = 3
IMG_SIZE = 224
BATCH_SIZE = 64
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training'
) 

Found 12092 images belonging to 3 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

Found 1510 images belonging to 3 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='categorical',
        #subset='validation'
)

Found 1509 images belonging to 3 classes.


In [7]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.DenseNet169(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model.summary()

___________
conv5_block21_0_relu (Activatio (None, 7, 7, 1280)   0           conv5_block21_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_1_conv (Conv2D)   (None, 7, 7, 128)    163840      conv5_block21_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block21_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block21_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block21_1_relu[0][0]       
_________________________________________________________________________________________________

In [8]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
188/188 [==============================] - 1259s 7s/step - loss: 0.9250 - accuracy: 0.7909 - precision: 0.7975 - recall: 0.7848 - val_loss: 0.4641 - val_accuracy: 0.8682 - val_precision: 0.8682 - val_recall: 0.8682
Epoch 2/5
188/188 [==============================] - 1096s 6s/step - loss: 0.3788 - accuracy: 0.8644 - precision: 0.8658 - recall: 0.8610 - val_loss: 0.3604 - val_accuracy: 0.8614 - val_precision: 0.8624 - val_recall: 0.8601
Epoch 3/5
188/188 [==============================] - 1098s 6s/step - loss: 0.3299 - accuracy: 0.8830 - precision: 0.8851 - recall: 0.8824 - val_loss: 0.3521 - val_accuracy: 0.8655 - val_precision: 0.8661 - val_recall: 0.8655
Epoch 4/5
188/188 [==============================] - 1110s 6s/step - loss: 0.3399 - accuracy: 0.8761 - precision: 0.8784 - recall: 0.8741 - val_loss: 0.3356 - val_accuracy: 0.8798 - val_precision: 0.8803 - val_recall: 0.8791
Epoch 5/5
188/188 [==============================] - 1086s 6s/step - loss: 0.3266 - accuracy: 0.8751

In [9]:
model.save('F:/Xray_Gaus/Densenet169_GausAd_wt')

history.history

INFO:tensorflow:Assets written to: F:/Xray_Gaus/Densenet169_GausAd_wt\assets


{'loss': [0.6152776479721069,
  0.3729001581668854,
  0.3435588479042053,
  0.33473509550094604,
  0.31990277767181396],
 'accuracy': [0.832391083240509,
  0.8668938875198364,
  0.8716328740119934,
  0.8774526119232178,
  0.8808613419532776],
 'precision': [0.8374989628791809,
  0.8689430356025696,
  0.8738422989845276,
  0.8793118000030518,
  0.882274329662323],
 'recall': [0.8274027109146118,
  0.8632357716560364,
  0.8707183003425598,
  0.8752909898757935,
  0.8785334229469299],
 'val_loss': [0.46406251192092896,
  0.3603901267051697,
  0.35210323333740234,
  0.3355707824230194,
  0.3054395616054535],
 'val_accuracy': [0.8682065010070801,
  0.8614130616188049,
  0.86548912525177,
  0.879755437374115,
  0.8851901888847351],
 'val_precision': [0.8682065010070801,
  0.8623978495597839,
  0.8660774827003479,
  0.8802720904350281,
  0.8863945603370667],
 'val_recall': [0.8682065010070801,
  0.8600543737411499,
  0.86548912525177,
  0.8790760636329651,
  0.8851901888847351]}

In [10]:
results = model.evaluate(test_gen, batch_size=128)

48/48 [==============================] - 109s 2s/step - loss: 0.2831 - accuracy: 0.8847 - precision: 0.8853 - recall: 0.8847


In [11]:
y_pred = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)

48/48 [==============================] - 110s 2s/step


In [12]:
y_true=test_gen.labels

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.59      0.59      0.59       885
           2       0.41      0.42      0.42       604

    accuracy                           0.52      1509
   macro avg       0.34      0.34      0.34      1509
weighted avg       0.51      0.52      0.51      1509



In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[  0,  13,   7],
       [  5, 523, 357],
       [  3, 345, 256]], dtype=int64)